In [ ]:
import gw_detections_functions as gwfn
import quantum_matched_filter_functions as qmffn
from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes, mark_inset
from scipy.interpolate import interp1d
import matplotlib.pyplot as plt
import matplotlib
import scipy.signal
import numpy as np

In [ ]:
path = './outpath/'
os.makedirs(path, exist_ok=True)

In [ ]:
!wget "https://www.gw-openscience.org/eventapi/html/GWTC-1-confident/GW150914/v3/H-H1_GWOSC_4KHZ_R1-1126259447-32.txt.gz"
!wget "https://dcc.ligo.org/public/0158/P1900011/001/GWTC1_GW150914_PSDs.dat"
!gzip -d "H-H1_GWOSC_4KHZ_R1-1126259447-32.txt.gz"
!mv "GWTC1_GW150914_PSDs.dat" "H-H1_GWOSC_4KHZ_R1-1126259447-32.txt" "data/."

In [ ]:
ht = np.loadtxt('data/H-H1_GWOSC_4KHZ_R1-1126259447-32.txt', skiprows=3)
psd_data = np.loadtxt('data/GWTC1_GW150914_PSDs.dat', skiprows=1, usecols=[0,1])

In [ ]:
fs = 4096
f_low = 20.
dt=1./fs
T = 32
df = 1./T
N = int(T*fs)
t = dt*np.arange(N)
f = df*np.arange(N//2 + 1)

In [ ]:
T_new = 16
fs_new = 4096
dt_new = 1./fs_new
df_new = 1./T_new
N_new = int(T_new*fs)
t_new = dt*np.arange(N_new)
f_new = df_new*np.arange(N_new//2 + 1)

In [ ]:
inv_psd_func = interp1d(psd_data[:,0],1.0/psd_data[:,1],kind='linear',fill_value=(0.0,0.0),bounds_error=False)
inv_psd = inv_psd_func(f)

In [ ]:
ht_w = gwfn.whiten(ht,inv_psd,fs)

In [ ]:
ht_wb = gwfn.highpass(ht_w,f_low,fs)

In [ ]:
ht_wbc = ht_wb[fs*(T - T_new)//2:-fs*(T - T_new)//2]

In [ ]:
inv_psd = inv_psd_func(f_new)

In [ ]:
#_, st_wbc = gwfn.make_template(54.89,18.66,fs_new,T_new,inv_psd)#,s1=-0.39,s2=0.89)
_, st_wbc = gwfn.make_template(35.6,30.6,fs_new,T_new,inv_psd)#,s1=-0.39,s2=0.89)

In [ ]:
snr = gwfn.get_snr(ht_wbc,st_wbc,fs_new)

In [ ]:
print(np.max(snr))

In [ ]:
plt.plot(snr)

In [ ]:
np.save('data/signal',ht_wbc)
np.save('data/psd', 1./inv_psd)

In [ ]:
plt.plot(ht_wbc)

In [ ]:
bank, _, _ = qmffn.get_paras(2**8)

In [ ]:
m1s = bank['mass1']
m2s = bank['mass2']
s1s = bank['spin1z']
s2s = bank['spin2z']
fls = bank['f_lower']
paras = iter(np.array([m1s,m2s,s1s,s2s,fls]).T)

In [ ]:
psd_func = interp1d(psd_data[:,0],psd_data[:,1],kind='linear',fill_value=(psd_data[0,1],psd_data[0,1]),bounds_error=False)
psd = psd_func(f_new)
plt.loglog(f_new,psd)

In [ ]:
#noise_ = pycbc.noise.noise_from_psd(len(strain), strain.delta_t, psd_noise)

In [ ]:
fontsize = 28
ticksize = 22

matplotlib.rcParams['mathtext.fontset'] = 'stix'
matplotlib.rcParams['font.family'] = 'STIXGeneral'

fig, ax = plt.subplots(figsize=(12,8))

temp = np.roll(st_wbc,np.argmax(ht_wbc)-np.argmax(st_wbc))

ax.plot(np.arange(len(ht_wbc))*dt,ht_wbc, color='black', lw=.2, label=r'$x(t)$')
ax.plot(np.arange(len(ht_wbc))*dt,np.max(ht_wbc)*temp/np.max(st_wbc), color='orange', lw=.2, label=r'$q(t)$')

ax.set_xlabel(r'$t$ (seconds)', fontsize=fontsize)
ax.set_ylabel(r'Whitened strain', fontsize=fontsize, labelpad=-10)
ax.tick_params(axis='both', labelsize=ticksize)
ax.set_ylim(-3.5,6)
ax.set_xlim(0,16)
ax.set_yticks([-2.,0,2])

axins = zoomed_inset_axes(ax, 70, loc=1)
axins.plot(np.arange(len(ht_wbc))*dt,ht_wbc/500, color='black', lw=2)
axins.plot(np.arange(len(ht_wbc))*dt,np.max(ht_wbc)*temp/(500*np.max(st_wbc)), color='orange', lw=2)
xmin, xmax = 7.2, 7.45
indmin, indmax = xmin/dt, xmax/dt
axins.set_xlim(xmin,xmax)
axins.set_xticks([])
axins.set_yticks([])
mark_inset(ax, axins, loc1=3, loc2=4, fc="none", ec="0.5")

#leg = ax.legend(fontsize=fontsize, loc='lower right', markerscale=50)

fig.savefig(path+'signal_zoom.png')

plt.show()

In [ ]:
plt.plot(np.arange(len(ht_wbc))*dt,np.max(ht_wbc)*st_wbc/np.max(st_wbc), color='black', lw=.2)
plt.show()

In [ ]:
import pycbc.noise
import pycbc.psd
import pylab

# The color of the noise matches a PSD which you provide
flen = int(2048 / df) + 1
noise_psd = pycbc.psd.aLIGOMidLowSensitivityP1200087(flen, df, f_low)

# Generate 32 seconds of noise at 4096 Hz
tsamples = int(32 / dt)
noise = pycbc.noise.noise_from_psd(tsamples, dt, noise_psd)

pylab.plot(noise.sample_times, noise)
pylab.ylabel('Strain')
pylab.xlabel('Time (s)')
pylab.show()

In [ ]:
plt.loglog(noise_psd.sample_frequencies,noise_psd)
plt.loglog(psd_data[:,0],psd_data[:,1])

plt.show()

In [ ]:
inv_noise_psd_func = interp1d(noise_psd.sample_frequencies,1.0/np.array(noise_psd),kind='linear',fill_value=(0.0,0.0),bounds_error=False)
inv_noise_psd = inv_noise_psd_func(f)
inv_noise_psd = np.where(inv_noise_psd==np.inf, 0., inv_noise_psd)

In [ ]:
print(inv_psd)
print(inv_noise_psd)

In [ ]:
noise_w = gwfn.whiten(np.array(noise),inv_noise_psd,fs)

In [ ]:
noise_wb = gwfn.highpass(noise_w,f_low,fs)

In [ ]:
noise_wbc = noise_wb[fs*(T - T_new)//2:-fs*(T - T_new)//2]

In [ ]:
inv_noise_psd_new = inv_noise_psd_func(f_new)
inv_noise_psd_new = np.where(inv_noise_psd_new==np.inf, 0., inv_noise_psd_new)

In [ ]:
_, st_wbc = gwfn.make_template(35.6,30.6,fs_new,T_new,inv_noise_psd_new)

In [ ]:
snr = gwfn.get_snr(noise_wbc,st_wbc,fs_new)
print(np.max(snr))

In [ ]:
np.save('data/noise',noise_wbc)
np.save('data/noise_psd', 1./inv_noise_psd_new)